In [ ]:
import sys
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

class idData:
	def __init__(self, id, app_data, chest, left, right):
		self.id = id
		self.app_data = app_data
		self.chest = chest
		self.left = left
		self.right = right

# return timing intervals as indicated in appData.json
# labeled in unix timestamps
def get_timings(app_data):
	stamps = app_data['activityTimestamps']
	timings = []
	for s in stamps:
		t1 = s['enter']
		t2 = s['leave']
		timings.append(t1)
		timings.append(t2)
	return timings

# return timing intervals as indicated in appData.json
# labeled in minutes
def get_timings_min(app_data):
	stamps = app_data['activityTimestamps']
	timings = []
	for s in stamps:
		t1 = ((s['enter']/1000)/60) % 60
		t2 = ((s['leave']/1000)/60) % 60
		timings.append(t1)
		timings.append(t2)
	return timings

def draw_timings(plt, timings):
	plt.axvspan(timings[4], timings[5], alpha=0.2, color='green') 	# First sliders
	plt.axvspan(timings[6], timings[7], alpha=0.3, color='blue') 	# First relax
	plt.axvspan(timings[8], timings[9], alpha=0.2, color='pink') 	# First record
	plt.axvspan(timings[10], timings[11], alpha=0.2, color='green')	# Second sliders
	plt.axvspan(timings[12], timings[13], alpha=0.2, color='red', label='MIST')	# MIST
	plt.axvspan(timings[14], timings[15], alpha=0.3, color='purple')# Second record
	plt.axvspan(timings[16], timings[17], alpha=0.4, color='pink')	# Feedback
	plt.axvspan(timings[18], timings[19], alpha=0.2, color='green') # Third sliders
	plt.axvspan(timings[20], timings[21], alpha=0.3, color='blue')	# Second relax
	plt.axvspan(timings[22], timings[23], alpha=0.2, color='green') # Fourth sliders
	plt.axvspan(timings[24], timings[25], alpha=0.25, color='green')# PANAS
	plt.axvspan(timings[26], timings[27], alpha=0.25, color='green')# DASS

# plot the EDA/GSR
# data is an idData instance
# scaling 'none', 'rest_normalized', 'normalized'
# no scaling plots GSR as is, 'normalized' scaled all data from 0 to 1
# and 'rest_normalized' sets the end of the first rest period to 1 with
# 0 at the lowest point before rest
def plot_GSR(data, scaling):
	print('Plotting GSR of id '+data.id)
	minutes = ((data.left['time']/1000)/60) % 60
	hours = (((data.left['time']/1000)/60)/60) % 24
	plt.figure(figsize=(15,8))
	plt.title('EDA of ID '+str(data.id),fontsize=22)
	plt.plot(data.left['time'], data.left['GSR_SCL'],linewidth=2.0,color='black')
	plt.plot(data.left['time'], data.right['GSR_SCL'],linewidth=2.0,color='blue')
	xt = []
	xl = []
	c = 0
	for m in minutes:
		if m % 5 == 0:
			xt.append((data.left['time'])[c])
			print(c)
			xl.append(str(int(hours[c]))+':'+str(int(m))) # hh:mm label
		c += 1
	plt.xticks(xt, xl)
	timings = get_timings(data.app_data)
	draw_timings(plt, timings)
	questions = (data.app_data['actualQuestions'])[0]
	plt.axvline(questions['start'], linestyle='--', color='black')
	plt.grid(True)
	plt.xlabel('Time of day (hours)', fontsize=18)
	plt.ylabel('GSR_SCL', fontsize=18)
	plt.show()
	return

# plot a statistical population plot of GSR
# id_datas and ids to be considered
def plot_pop_GSR(id_datas, ids):
	all_data = pd.DataFrame()
	lengths = []
	longest_min = []
	for id in ids:
		gsr = id_datas[id].left

		# Cut GSR from beginning of first rest to end of PANAS
		minutes = ((gsr['time']/1000)/60) % 60
		timings = get_timings_min(id_datas[id].app_data)
		c = 0
		for m in minutes:
			if abs(timings[6]-m) <= 0.015: # Less than a second time difference
				start = c
			if abs(timings[25]-m) <= 0.015:
				end = c
			c += 1
		gsr = gsr[start:end]
		if len(minutes) > len(longest_min):
			longest_min = minutes-minutes[0] # Use a relative timescale, as long as the longest timeseries
		i = gsr['time'].index.values[0]
		gsr['time'] = gsr['time'] - (gsr['time'])[i]
		all_data = all_data.append(gsr)
		lengths.append(len(gsr))

	# Examine difference in GSR data length
	plt.hist(lengths)
	plt.title('Lengths of cut GSR data')
	plt.show()

	# Line plot with confidence interval
	all_data.set_index('time')
	ax = sns.lineplot(x='time', y='GSR_SCL', data=all_data)
	plt.xlabel('Time (minutes)', fontsize=18)
	plt.ylabel('GSR_SCL', fontsize=18)
	plt.grid(True)
	plt.show()

# Load all data

id_datas = {}	# Contains data objects of all valid IDs (own an appData file)
chests = []		# IDs who own chest.csv
lefts = []		# IDs who own left.csv
rights = [] 	# IDs who own right.csv
completes = []	# IDs who own all of the above

dir_names = next(os.walk('.'))[1]
print('Found '+str(len(dir_names))+' IDs.')

# Loop folders
for id in dir_names:
	if os.path.isfile(os.path.join(id, 'appData.json')):
		with open(os.path.join(id,'appData.json')) as app_file:
			app_data_string = app_file.read()
			app_data = json.loads(app_data_string)
			data = idData(id, app_data, 'none', 'none', 'none')
		id_datas[id] = data

		# Check for and load CSV files
		complete = True
		if os.path.isfile(os.path.join(id, 'chest.csv')):
			chest = pd.read_csv(os.path.join(id, 'chest.csv'), sep=',')
			data.chest = chest
			chests.append(id)
		else:
			complete = False
		if os.path.isfile(os.path.join(id, 'left.csv')):
			left = pd.read_csv(os.path.join(id, 'left.csv'), sep=',')
			data.left = left
			lefts.append(id)
		else:
			complete = False
		if os.path.isfile(os.path.join(id, 'right.csv')):
			right = pd.read_csv(os.path.join(id, 'right.csv'), sep=',')
			data.right = right
			rights.append(id)
		else:
			complete = False
		if complete:
			completes.append(id)
		
print(str(len(id_datas))+' IDs contained appData.json')
print(str(len(completes))+' IDs contained all chest and bands data')

print('')
print("Commands: plot (GSR|ECG) (ID|'all') ('none'|'rest_normalized'|'normalized')\n")

# Command loop for CLI
while True:
	x = input()
	cmd = x.split()
	
	if cmd[0].lower() == 'plot':
		if len(cmd) == 3:
			cmd.append('none')
		elif len(cmd) < 3:
			print('Not enough arguments')
			continue
		if cmd[1].lower() == "gsr":
			if cmd[2].lower() == 'all':
				# Plot population GSR
				plot_pop_GSR(id_datas, lefts)
			else:
				id = cmd[2]
				# Plot GSR
				if (id in lefts) or (id in rights):
					plot_GSR(id_datas[id], cmd[3])